## Task 2 :detect faces (no eyes) and with the optimization.ipynb
Use Haar Cascades to dectect

Import necessary libraries

In [ ]:
#Use Javascript code to call your local webcam
from IPython.display import display, Javascript
from google.colab.output import eval_js
#Convert the image type that captured by webcam
from base64 import b64decode, b64encode

import numpy as np
from PIL import Image

import io
import cv2 # OpenCV library 

JavaScript Code to call local WebCam and the image type is base64

In [ ]:
from google.colab.output import eval_js

def VideoCapture():
  js = Javascript('''
    async function create(){
      div = document.createElement('div');
      document.body.appendChild(div);

      video = document.createElement('video');
      video.setAttribute('playsinline', '');

      div.appendChild(video);
      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "environment"}});
      video.srcObject = stream;

      await video.play();

      canvas =  document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      div_out = document.createElement('div');
      document.body.appendChild(div_out);
      img = document.createElement('img');
      div_out.appendChild(img);
    }

    async function capture(){
        return await new Promise(function(resolve, reject){
            pendingResolve = resolve;
            canvas.getContext('2d').drawImage(video, 0, 0);
            result = canvas.toDataURL('image/jpeg', 0.40);

            pendingResolve(result);
        })
    }

    function showimg(imgb64){
        img.src = "data:image/jpg;base64," + imgb64;
    }

  ''')
  display(js)

### Define the Image Conversion function

For VideoCapture functions return images with base64 type

For cv2 need images with bytes type

In [ ]:
def b64_to_bytes(byte):
  jpeg = b64decode(byte.split(',')[1])
  im = Image.open(io.BytesIO(jpeg))
  return np.array(im)

def bytes_to_b64(image):
  image = Image.fromarray(image)
  buffer = io.BytesIO()
  image.save(buffer, 'jpeg')
  buffer.seek(0)
  x = b64encode(buffer.read()).decode('utf-8')
  return x

The haar cascades files are in Google drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Load cascades using cv2.CascadeClassifier
faces_cascades = cv2.CascadeClassifier("/content/drive/MyDrive/Computer_Vision/haarcascade_frontalface_default.xml")
eyes_cascade = cv2.CascadeClassifier("/content/drive/MyDrive/Computer_Vision/haarcascade_eye.xml")

In [ ]:
def detect_faces(img, cascades):
  #transform image into grayscale
  gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  #use cascades to detect faces
  faces = cascades.detectMultiScale(gray, 1.3, 4)
  return faces

### Compute the sub region

In [ ]:
#This function computes the sub region in which we want to detect faces
def compute_optimized_search_region(previous, img_height, img_width, margin):
  # here previous is the array of faces (x of top left, y of top left, face width, face height)
  # img_height and img_width is the whole img captured by the webcam, we use this to avoid the bounding box over the image
  # margin is the distance between bounding box and face region
  x_tl, y_tl = previous[0] - margin, previous[1] - margin
  x_br, y_br = previous[0] + previous[2] + margin, previous[1] + previous[3] + margin
  #set bounding box constraints
  x_tl, y_tl = max(x_tl, 0), max(y_tl, 0)
  x_br, y_br = min(x_br, img_width), min(y_br, img_height)
  # return the new bounding box coordinates
  return (x_tl, y_tl, x_br - x_tl, y_br - y_tl)

In [ ]:
VideoCapture()
eval_js('create()')# call the local camera to capture

b64_im = eval_js('capture()') # Take a capture 
im = b64_to_bytes(b64_im) # Convert the capture brom base64 to bytes array
# get the whole image height and width
(img_height, img_width) = im.shape[0], im.shape[1]
# set the previous box as none
previous_bbox = None
margin = 50

while True:

  b64_im = eval_js('capture()') # Take a capture 
  im = b64_to_bytes(b64_im) # Convert the capture brom base64 to bytes array
  region_to_use = im

  if previous_bbox is not None:
    (new_x,new_y,new_width,new_height) = compute_optimized_search_region(previous_bbox,img_height,img_width,margin)
    #retrieve the sub region
    region_to_use = im[new_y:new_y+new_height, new_x:new_x+new_width]
    #draw a red rectangle
    cv2.rectangle(im, (new_x,new_y), (new_x+new_width, new_y+new_height),(255,0,0),5)

  #Detect faces in subregion
  faces = detect_faces(region_to_use, faces_cascades)
  
  if len(faces) == 1:
    #update the face bounding box
    face = faces[0]
    # here, if the previous box is not None which means the region we searched is the bounding box.
    # so if we want to find the correct coordinates of face on the whole image, we need to add the bounding box coordinates
    (x,y,w,h) = face if previous_bbox is None else (new_x+face[0], new_y+face[1], face[2], face[3])
    # update previous_bbox
    previous_bbox = (x,y,w,h)
    #draw a green rectangle around the detected face
    cv2.rectangle(im, (x,y), (x+w, y+h), (0, 255, 0), 2)

    # here if we can find the face region, we also could find eyes in the face region
    face_roi_im = im[y:y+h, x:x+w]
    face_roi_gray = cv2.cvtColor(face_roi_im,cv2.COLOR_BGR2GRAY)
    
    # detect eyes
    eyes = eyes_cascade.detectMultiScale(face_roi_gray, 1.3, 4)
    for(e_x,e_y,e_w,e_h)in eyes:
      #draw a rectangle around each eye
      cv2.rectangle(face_roi_im,(e_x,e_y),(e_x+e_w,e_y+e_h),(0,0,255),2)

  else:
    # currently we assume only one face in  the image
    previous_bbox= None

  #convert image with bounding boxes into base64 and plot it using JS
  eval_js('showimg("{}")'.format(bytes_to_b64(im)))

<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored

In [ ]:
im.shape